In [23]:
import os
import math
import json
import librosa
from librosa import feature
from os.path import join as pjoin

In [25]:
current_dir = os.getcwd()
data_dir = pjoin(current_dir, 'Data')
song_path = pjoin(data_dir, 'genres_original')

DATASET_PATH = song_path
JSON_PATH = "data.json"
SAMPLE_RATE = 22050
DURATION = 30  # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segements=5):

    # Dictionary to store data
    data = {
        "mapping": [],  # genre labels in numbers
        "mfcc": [],     # mfcc for each segment
        "labels": []    # targets we expect
    }

    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segements)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)

    # Loop through all genres
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # Ensure we are not at the root level
        if dirpath != dataset_path:

            # Save the semantic label
            dirpath_components = dirpath.split(os.sep)
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing {}".format(semantic_label))

            # Process files for a specific genre
            for f in filenames:

                # Load audio file
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

                # Process segments extracting mfcc and storing data
                for s in range(num_segements):
                    start_sample = num_samples_per_segment * s
                    finish_sample = start_sample + num_samples_per_segment
                    mfcc_features = librosa.feature.spectral.mfcc(signal[start_sample:finish_sample],
                                                                  sr=sr, n_fft=n_fft, n_mfcc=n_mfcc,
                                                                  hop_length=hop_length)
                    mfcc_features = mfcc_features.T
                    # Store mfcc for segment if it has expected length
                    if len(mfcc_features) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc_features.tolist())
                        data["labels"].append(i - 1)
                        print("{}, segment:{}".format(file_path, s))
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)


In [26]:
if __name__ == "__main__":
    save_mfcc(DATASET_PATH, JSON_PATH, num_segements=10)


Processing pop


TypeError: mfcc() takes 0 positional arguments but 1 positional argument (and 2 keyword-only arguments) were given